In [54]:
import numpy as np
import cmath
from time import time

**The Riemann Zeta function is defined as:**
$$\zeta(s):=\sum_{n=1}^\infty{n^{-s}} \ \ \ \ : \ (s=\sigma + it) \ \ ; \ (\sigma > 1)$$

**Computations of the Riemann Zeta function on the critical line refer to computations of the form $\zeta(\frac{1}{2} + it)$, where we move vertically along the complex plane with $t$ while being fixed at the point $\frac{1}{2}$ on the real number line.** 

In [94]:
# naive implementation of the Riemann Zeta Function 
def zetaN(x: complex): return sum([1/(i**x) for i in range(1, int(1e7))])

In [95]:
sample_point = complex(real=0.5, imag=14.1347)

In [96]:
sample_point

(0.5+14.1347j)

In [97]:
start = time.time()

In [98]:
zetaN(sample_point)

(222.5904117858588-21.061202682724637j)

In [99]:
start - time.time()

-10.316173076629639